In [1]:
import torch
import matplotlib.pyplot as plt
import torchvision
import cv2
import glob

In [17]:
#Link categories
def link_categories(cat_id):
    if cat_id == 1:
        return "Cell Phone"
    elif cat_id == 2:
        return "Face"
    elif cat_id == 3:
        return "Hand"
    elif cat_id == 4:
        return "OOD"
    elif cat_id == 5:
        return "Cell Phone Attached"

In [18]:
def lookup_annotations(image_name, coco):
    image_id = None
    for image in coco['images']:
        if image['file_name'] == image_name:
            image_id = image['id']
            break
    if image_id is None:
        return []
    return [annotation for annotation in coco['annotations'] if annotation['image_id'] == image_id]

In [21]:
def calculate_distance(point, point2):
    px, py = point
    x, y = point2
    return ((px - x) ** 2 + (py - y) ** 2) ** 0.5

In [24]:
def phone_use(people, phones):
    phone_use = {}
    for idx, phone in phones.items():
        distance = 1000000
        for P_idx, person in people.items():
            new_distance = calculate_distance((person["bbox"][0]+person["bbox"][3]/2, person["bbox"][1]+person["bbox"][4]/2), (phone["bbox"][0]+phone["bbox"][3]/2, phone["bbox"][1]+phone["bbox"][4]/2))
            if new_distance < distance:
                distance = new_distance
                phone_use[idx] = p_idx

    return phone_use

In [20]:
def run_yolo(model,image_path, coco):
    """
    ids = {
        CIH__id: 0,
        Cell: 4,
        face_id: 1,
    }
    """
    people = {}
    driver = None
    phones = {}
    hands = {}
    image_data = None
    for image in coco['images']:
        if image["file_name"] == image_path:
            image_data = image.copy()
            break

    annotations = lookup_annotations(image_path, coco)
    num_faces = 0
    phone_cnt = 0
    hand_cnt = 0

    for annotation in annotations:
        if annotation['category_id'] == 2:
            people[num_faces] = annotation
            num_faces += 1
        elif annotation["category_id"] == 3:
            annotation['category_id'] = "Hand"
            hands[hand_cnt] = annotation
            hand_cnt += 1
        elif annotation["category_id"] == 1:
            annotation['category_id'] = "Phone"
            phones[phone_cnt]
            phone_cnt += 1

    phone_use_list = phone_use(people, phones)

    for annotation in annotations:
        if annotation["category_id"] == 2:
            if annotation["area"] < 5000 and num_faces > 1:
                annotation['category_id'] = "Passenger"
            elif annotation["bbox"][0] < image_data["width"] / 2 and num_faces > 1:
                annotation['category_id'] = "Passenger"
            else:
                annotation['category_id'] = "Driver"
                driver = annotation

    

        
        

    PASSENGER_COLOR = (122, 48, 231)
    PASSENGER_USE = (1, 255, 4)
    HAND = (101, 124, 158)
    DRIVER = (34, 45, 126)
    DRIVER_USE = (252, 3, 0)
    THICKNESS = 2
    FONT_SCALE = 0.5
    
    results = model(image_path)

    cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)
    (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 1)
    cv2.rectangle(image, (x, y - text_height - baseline), (x + text_width, y), color, thickness=cv2.FILLED)
    cv2.putText(image, label, (x, y - baseline), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), 1, cv2.LINE_AA)

    results.save()
    
    return results

In [7]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath


# To run the model You just need to clone yolov5 repository from github 
# and add new_yolo_2.pt into yolov5\models\ directory
# https://github.com/ultralytics/yolov5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torch.hub.load(r"yolov5", 'custom', path=r"yolov5\models\new_yolo_2.pt", source='local') 

#model.conf = 0.50  # confidence threshold (0-1)
#model.iou = 0.45  # NMS IoU threshold (0-1)  

model = model.to(device)

YOLOv5  v7.0-330-gb20fa802 Python-3.11.9 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 212 layers, 20869098 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [8]:
result = run_yolo(model, 'PREPROCESSED_IMAGES/0/Image921.jpg')

Saved 1 image to runs\detect\exp


In [16]:
result.pred[0]

tensor([[766.24011, 303.59766, 859.07654, 431.16425,   0.95058,   1.00000],
        [687.71204, 468.83743, 783.04309, 541.87195,   0.90416,   2.00000]])

In [ ]:
# Class Names: ["Cell Phone", "Face", "Hand", "OOD", "Cell Phone Attached"]

results = {}
ids = {
    "CIH__id": 0,
    "Cell": 4,
    "face_id": 1,
}

status = ["0","1"]
labels,inputs = [], []

for i,s in enumerate(status):
    for index,image_path in enumerate(glob.glob("BLURRED_IMAGES\\"+s+"\\*")):
        """
        PART 2: FILL THIS AREA
        """


In [ ]:
def run_yolo_hand_Extraction(model,image_path,ids):
    """
    ids = {
        CIH__id: 0,
        Cell: 4,
        face_id: 1,
    }
    """
    
    results = model(image_path)

    """
    PART 3: FILL THIS AREA
    """
        
    return 0


status = ["1","0"]
labels,inputs = [], []

ids = {
    "CIH__id": 0,
    "Cell": 4,
    "face_id": 1,
}

for i,s in enumerate(status):
    for index,image_path in enumerate(glob.glob("BLURRED_IMAGES\\"+s+"\\*")):
        """
        PART 3: FILL THIS AREA
        """
